## Setup

Für die Aufgabe wird die Bibliothek "pandas" benötigt. Außerdem importieren wir nltk.

In [3]:
import pandas as pd
import nltk


Wir importieren unsere CSV-Datei aus Aufgabe 1

In [5]:
df = pd.read_csv("../data/raw/webscraping.csv")

Nun lassen wir uns den Dataframe ausgeben.

In [6]:
pd.read_csv(df)

TypeError: argument of type 'method' is not iterable

In [7]:
df['inhalt'] = df['inhalt'].astype(str).str.lower()

df.head(3)

KeyError: 'inhalt'

## Stopwords

wir laden uns aus nltk die "stopwords" herunter. Das sind Wörter in einer Stoppliste, die vor der Analyse von natürlichsprachlichen Daten gelöscht werden, da sie keine wertvollen Informationen enthalten.

In [ ]:
nltk.download('stopwords')

In [ ]:
import nltk
from nltk.corpus import stopwords

Wir speichern die Stoppwörter in "stopwords". Außerdem wollen wir nur die englischen Stoppwörter ausgeben.

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")


In [ ]:
stopwords

In [ ]:
df['inhalt_token_s'] = df['inhalt_token'].apply(lambda x: [i for i in x if i not in stopwords])